# Generate simulated infrastructure telemetry 

In [ ]:
# Install requiered packages if needed (only once)
!pip install pytimeparse
!pip install -i https://test.pypi.org/simple/ v3io-generator --upgrade
!pip install faker

In [1]:
import os
import pandas as pd
import time
import v3io_frames as v3f
import yaml
from v3io_generator import metrics_generator, deployment_generator

## Create Metadata
the following section will create a list of devices which are scattered in multiple datacenters

In [2]:
# Create meta-data factory
dep_gen = deployment_generator.deployment_generator()
faker=dep_gen.get_faker()

# Design meta-data
dep_gen.add_level(name='company',number=2,level_type=faker.company)
dep_gen.add_level('data_center',number=2,level_type=faker.street_name)
dep_gen.add_level('device',number=2,level_type=faker.msisdn)

# Create meta-data
deployment_df = dep_gen.generate_deployment()

Write the device list to a key/value table

In [3]:
client = v3f.Client('framesd:8081')

In [4]:
# uncomment the line below if you want to reset the key/value table 
#client.delete(backend='kv', table='netops_devices')

In [5]:
client.write(backend='kv', table='netops_devices',dfs=deployment_df, index_cols=['device'])

In [6]:
# verify the table is written 
client.read(backend='kv', table='netops_devices')

,company,data_center
__name,,
5728297877286,Boyd-Faulkner,Barrett_Crossing
8566530842848,Boyd-Faulkner,Barrett_Crossing
7542919702594,Boyd-Faulkner,Nancy_Mall
1184469177528,Wheeler__Shelton_and_Hill,Dean_Park
3916108545629,Boyd-Faulkner,Nancy_Mall
2368397418989,Wheeler__Shelton_and_Hill,Dean_Park
0847959813128,Wheeler__Shelton_and_Hill,Cindy_Rapids
4963294562966,Wheeler__Shelton_and_Hill,Cindy_Rapids


## Add initial values

In [7]:
deployment_df['cpu_utilization'] = 70
deployment_df['latency'] = 0
deployment_df['packet_loss'] = 0
deployment_df['throughput'] = 290
deployment_df.head()

,company,data_center,device,cpu_utilization,latency,packet_loss,throughput
0,Wheeler__Shelton_and_Hill,Dean_Park,2368397418989,70,0,0,290
1,Wheeler__Shelton_and_Hill,Dean_Park,1184469177528,70,0,0,290
2,Wheeler__Shelton_and_Hill,Cindy_Rapids,0847959813128,70,0,0,290
3,Wheeler__Shelton_and_Hill,Cindy_Rapids,4963294562966,70,0,0,290
4,Boyd-Faulkner,Barrett_Crossing,8566530842848,70,0,0,290


## Generate simulated metrics per device
Metrics schema (describe simulated values) is read from `metrics_configuration.yaml`

In [8]:
# Load metrics configuration from YAML file
with open('metrics_configuration.yaml', 'r') as f:
    metrics_configuration = yaml.load(f)

# Create metrics generator based on YAML configuration
met_gen = metrics_generator.Generator_df(metrics_configuration, user_hierarchy=deployment_df, initial_timestamp=time.time())
metrics = met_gen.generate()


In [9]:
num_samples = 1000
df = pd.concat([next(metrics) for i in range(num_samples)])
df.head(20)

cpu_utilization  \
timestamp                  data_center      device        company                                      
2019-03-05 14:38:47.338574 Dean_Park        2368397418989 Wheeler__Shelton_and_Hill            62.70   
                                            1184469177528 Wheeler__Shelton_and_Hill            67.19   
                           Cindy_Rapids     0847959813128 Wheeler__Shelton_and_Hill            73.02   
                                            4963294562966 Wheeler__Shelton_and_Hill            61.16   
                           Barrett_Crossing 8566530842848 Boyd-Faulkner                        78.61   
                                            5728297877286 Boyd-Faulkner                        85.61   
                           Nancy_Mall       3916108545629 Boyd-Faulkner                        55.59   
                                            7542919702594 Boyd-Faulkner                        66.63   
2019-03-05 14:38:53.338574 Dean_Park        2368397418989 Wheeler__Shelton_and_Hill            65.12   
                                            1184469177528 Wheeler__Shelton_and_Hill            96.60   
                           Cindy_Rapids     0847959813128 Wheeler__Shelton_and_Hill            50.61   
                                            4963294562966 Wheeler__Shelton_and_Hill            97.90   
                           Barrett_Crossing 8566530842848 Boyd-Faulkner                        74.27   
                                            5728297877286 Boyd-Faulkner                        66.44   
                           Nancy_Mall       3916108545629 Boyd-Faulkner                        82.64   
                                            7542919702594 Boyd-Faulkner                        83.28   
2019-03-05 14:38:59.338574 Dean_Park        2368397418989 Wheeler__Shelton_and_Hill            81.86   
                                            1184469177528 Wheeler__Shelton_and_Hill            63.77   
                           Cindy_Rapids     0847959813128 Wheeler__Shelton_and_Hill            61.60   
                                            4963294562966 Wheeler__Shelton_and_Hill            66.78   

                                                                                     cpu_utilization_is_error  \
timestamp                  data_center      device        company                                               
2019-03-05 14:38:47.338574 Dean_Park        2368397418989 Wheeler__Shelton_and_Hill                     False   
                                            1184469177528 Wheeler__Shelton_and_Hill                     False   
                           Cindy_Rapids     0847959813128 Wheeler__Shelton_and_Hill                     False   
                                            4963294562966 Wheeler__Shelton_and_Hill                     False   
                           Barrett_Crossing 8566530842848 Boyd-Faulkner                                 False   
                                            5728297877286 Boyd-Faulkner                                 False   
                           Nancy_Mall       3916108545629 Boyd-Faulkner                                 False   
                                            7542919702594 Boyd-Faulkner                                 False   
2019-03-05 14:38:53.338574 Dean_Park        2368397418989 Wheeler__Shelton_and_Hill                     False   
                                            1184469177528 Wheeler__Shelton_and_Hill                     False   
                           Cindy_Rapids     0847959813128 Wheeler__Shelton_and_Hill                     False   
                                            4963294562966 Wheeler__Shelton_and_Hill                     False   
                           Barrett_Crossing 8566530842848 Boyd-Faulkner                                 False   
                                            5728297877286 Boyd-Faulkner                              

## Save to Iguazio Time-series Database

In [10]:
# uncomment the line below if you want to reset the TSDB table 
#client.delete(backend='tsdb', table='netops_metrics_jupyter')

In [11]:
# create a new table, need to specify estimated sample rate
client.create(backend='tsdb', table='netops_metrics_jupyter', attrs={'rate': '1/m'})

In [12]:
# write the dataframe into the time-seried DB, note the company,data_center,device indexes are automatically converted to search optimized labels
client.write(backend='tsdb', table='netops_metrics_jupyter', dfs=df)

## Verify that the data was written

In [13]:
client.read(backend='tsdb', query='select avg(cpu_utilization) as cpu_utilization, avg(latency) as latency, avg(packet_loss) as packet_loss, avg(throughput) as throughput from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='5m').head(10)

,,,,cpu_utilization,latency,packet_loss,throughput
time,data_center,device,company,,,,
2019-03-05 14:34:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,71.093333,1.0800,0.111111,254.581111
2019-03-05 14:39:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,69.523200,2.2654,0.700000,251.579400
2019-03-05 14:44:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,82.449600,31.3854,17.600000,194.672200
2019-03-05 14:49:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,72.881800,13.5588,6.460000,218.545000
2019-03-05 14:54:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,83.358000,49.1876,23.380000,159.500800
2019-03-05 14:59:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,81.381800,35.3046,15.820000,186.931000
2019-03-05 15:04:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,73.243800,13.2330,6.620000,220.357800
2019-03-05 15:09:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,70.210000,1.7898,0.780000,251.933600
2019-03-05 15:14:37,Barrett_Crossing,8566530842848,Boyd-Faulkner,72.398200,2.1098,0.600000,248.303800


### Save the generated dataset to parquet for future reproducability 

In [15]:
# craete directory if doesnt exist 
!mkdir data

In [16]:
import pyarrow as pa

In [ ]:
#write the dataframe into a parquet (on iguazio file system)
version = '1.0'
filepath = 'data/netops_metrics.v{}.parquet'.format(version)
pa.parquet.write_table(pa.Table.from_pandas(df), filepath)

### Reading the data from parquet into the time-series DB
if we want to reproduce the same results we can rebuild the TSDB from the saved parquet file

In [16]:
# uncomment the line below if you want to reset the TSDB table 
#client.delete(backend='tsdb', table='netops_metrics_jupyter')
#client.create(backend='tsdb', table='netops_metrics_jupyter', attrs={'rate': '1/m'})

In [14]:
# read the parquet into memory and print the head 
pqdf = pa.parquet.read_table(filepath).to_pandas()
pqdf.head()

cpu_utilization  \
timestamp                  company                     data_center     device                           
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828            73.94   
                                                                       8654196541788            73.78   
                                                       Mccoy_Union     5082949640375            64.45   
                                                                       3879474364771            77.60   
                           Robinson__Andrews_and_Smith Mills_Fields    1569394356868            60.26   

                                                                                      cpu_utilization_is_error  \
timestamp                  company                     data_center     device                                    
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828                     False   
                                                                       8654196541788                     False   
                                                       Mccoy_Union     5082949640375                     False   
                                                                       3879474364771                     False   
                           Robinson__Andrews_and_Smith Mills_Fields    1569394356868                     False   

                                                                                      latency  \
timestamp                  company                     data_center     device                   
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828     0.00   
                                                                       8654196541788     0.00   
                                                       Mccoy_Union     5082949640375     0.00   
                                                                       3879474364771     0.00   
                           Robinson__Andrews_and_Smith Mills_Fields    1569394356868     0.43   

                                                                                      latency_is_error  \
timestamp                  company                     data_center     device                            
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828             False   
                                                                       8654196541788             False   
                                                       Mccoy_Union     5082949640375             False   
                                                                       3879474364771             False   
                           Robinson__Andrews_and_Smith Mills_Fields    1569394356868             False   

                                                                                      packet_loss  \
timestamp                  company                     data_center     device                       
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828          0.0   
                                                                       8654196541788          1.0   
                                                       Mccoy_Union     5082949640375          0.0   
                                                                       3879474364771          0.0   
                           Robinson__Andrews_and_Smith Mills_Fields    1569394356868          2.0   

                                                                                      packet_loss_is_error  \
timestamp                  company                     data_center     device                                
2019-02-23 14:07:28.219701 Brennan-Meyer               Bender_Mountain 8056353454828                 False   
                                                                       8654196541788      

In [17]:
# write the dataframe into the time-seried DB, uncomment the line below
#client.write(backend='tsdb', table='netops_metrics_jupyter', dfs=pqdf)